In [51]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import sidetable as stb

from scipy.stats import skew
from scipy.stats import kurtosistest
from scipy import stats
from scipy.stats import kstest

from datetime import datetime, timedelta

from datetime import date
import holidays 

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [52]:
df = pd.read_csv('data/bikes.csv', index_col = 0).reset_index(drop = True)
df.head(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 91.4+ KB


In [54]:
# Vemos las filas y columnas que tenemos:

df.shape

(730, 16)

In [55]:
# Visualizamos que no tenemos nulos:

df.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [56]:
# Comprobamos que no hay duplicados:

df.duplicated().sum()

0

In [57]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
instant,730.0,365.500000,210.877136,1.000000,183.250000,365.500000,547.750000,730.000000
yr,730.0,0.500000,0.500343,0.000000,0.000000,0.500000,1.000000,1.000000
mnth,730.0,6.526027,3.450215,1.000000,4.000000,7.000000,10.000000,12.000000
holiday,730.0,0.028767,0.167266,0.000000,0.000000,0.000000,0.000000,1.000000
weekday,730.0,2.997260,2.006161,0.000000,1.000000,3.000000,5.000000,6.000000
workingday,730.0,0.683562,0.465405,0.000000,0.000000,1.000000,1.000000,1.000000
weathersit,730.0,1.394521,0.544807,1.000000,1.000000,1.000000,2.000000,3.000000
temp,730.0,20.319259,7.506729,2.424346,13.811885,20.465826,26.880615,35.328347
atemp,730.0,23.726322,8.150308,3.953480,16.889713,24.368225,30.445775,42.044800
hum,730.0,62.765175,14.237589,0.000000,52.000000,62.625000,72.989575,97.250000


In [58]:
lista_columnas = list(df.columns)
lista_columnas

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

In [59]:
#cambiamos el nombre de las columnas para que sea más comprensible.

dicc_columnas = {'instant' : 'momento',
                'dteday' : 'fecha',
                'season' : 'estacion',
                'yr' : 'año',
                'mnth' : 'mes',
                'holiday' : 'festivo',
                'weekday' : 'dia_semana',
                'workingday' : 'laboral',
                'weathersit' : 'clima',
                'temp' : 'temperatura',
                'atemp' : 'sensacion_termica',
                'hum' : 'humedad',
                'windspeed' : 'viento',
                'casual' : 'casual',
                'registered' : 'registrado',
                'cnt' : 'total'}

df.rename(columns = dicc_columnas, inplace = True)

In [60]:
columnas_unicos = ['estacion', 'año', 'mes', 'festivo', 'dia_semana', 'laboral', 'clima']

In [61]:
# Revisamos los valores únicos para las columnas seleccionadas.

for col in columnas_unicos:
    print(f'La columna {col} tiene como valores únicos: {", ".join(map(str, df[col].unique()))}.')

La columna estacion tiene como valores únicos: spring, summer, autumn, winter.
La columna año tiene como valores únicos: 0, 1.
La columna mes tiene como valores únicos: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12.
La columna festivo tiene como valores únicos: 0, 1.
La columna dia_semana tiene como valores únicos: 6, 0, 1, 2, 3, 4, 5.
La columna laboral tiene como valores únicos: 0, 1.
La columna clima tiene como valores únicos: 2, 1, 3.


In [62]:
# Cambiamos el formato de esta columna a datetime:

df["fecha"] = df["fecha"].apply(pd.to_datetime)

/tmp/ipykernel_19478/2154209664.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["fecha"] = df["fecha"].apply(pd.to_datetime)


In [63]:
# Modificamos la columna de años ya que únicamente había 0 y 1.

df["año"]= df["fecha"].apply(lambda x: x.strftime("%Y"))


In [64]:
df.sample(5)

,momento,fecha,estacion,año,mes,festivo,dia_semana,laboral,clima,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
429,430,2019-06-03,spring,2019,3,0,2,1,1,10.591653,12.75210,45.6667,13.458625,221,3735,3956
502,503,2019-05-18,summer,2019,5,0,5,1,1,23.130847,27.55605,52.3333,9.166739,1521,6118,7639
376,377,2019-12-01,spring,2019,1,0,4,1,2,15.682500,19.09690,80.2917,12.124789,269,3828,4097
188,189,2018-08-07,autumn,2018,7,0,5,1,2,29.075847,33.52415,75.7917,15.083643,692,3348,4040
269,270,2018-09-27,winter,2018,9,0,2,1,2,26.103347,28.72625,88.5417,7.917457,477,3643,4120


In [65]:
# Nos hemos dado cuenta de que los meses de la columna "mes" no coinciden con los de "fecha". Por tanto, vamos a ajustarlos según el mes indicado fecha.

df["mes"] = df["fecha"].dt.strftime("%B")
df["mes"].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [66]:
meses = {"January": "enero", "February" : "febrero", "March" : "marzo", "April" : "abril", "May" : "mayo", "June" : "junio", "July" : "julio",
       "August": "agosto", "September" : "septiembre", "October" : "octubre", "November" : "noviembre", "December" : "diciembre"}


df["mes"] = df["mes"].map(meses)

In [67]:
df["mes"].unique()

array(['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio',
       'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre'],
      dtype=object)

In [68]:
# Nos hemos dado cuenta de que las estaciones están cambiadas, así que vamos a ponerlas bien.

def cambiar_estacion (fecha):

    """Recibe una columna en formato datetime, y en función del mes y el día devuelve la estación del año a la que pertenece"""
    
    mes = fecha.strftime("%B")
    dia = int(fecha.strftime("%d"))
    
    if mes == "April" or mes == "May":
        return "primavera"
    
    elif (mes == "March" and dia >= 20) or (mes == "June" and dia <= 20):
        return "primavera"
    
    if mes == "July" or  mes == "August":
        return "verano"
    
    elif (mes == "June" and dia >= 21) or (mes == "September" and dia <= 22):
        return "verano"
    
    if mes == "October" or mes == "November":
        return "otoño"
    
    elif (mes == "September" and dia >= 23) or (mes == "December" and dia <= 21):
        return "otoño"
    else:
        return "invierno"


In [69]:
df["estacion"] = df["fecha"].apply(cambiar_estacion)

In [70]:
df["estacion"].unique()

array(['invierno', 'primavera', 'verano', 'otoño'], dtype=object)

In [72]:
#También están mal los días de la semana, así que los cambiamos. 

df["dia_semana"] = df["fecha"].dt.strftime("%w")

,momento,fecha,estacion,año,mes,festivo,dia_semana,laboral,clima,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
661,662,2019-10-24,otoño,2019,octubre,0,4,1,1,24.087500,27.93605,63.6250,4.876206,795,6898,7693
85,86,2018-03-27,primavera,2018,marzo,0,2,0,2,10.374763,12.51695,49.3913,12.348100,472,1221,1693
639,640,2019-02-10,invierno,2019,febrero,0,0,1,3,24.224153,27.11665,87.1667,6.999825,315,4324,4639
285,286,2018-10-13,otoño,2018,octubre,0,6,1,2,24.155847,27.59020,89.6667,9.499729,290,2623,2913
359,360,2018-12-26,invierno,2018,diciembre,1,3,0,1,13.191299,15.77675,50.6957,16.044155,430,887,1317
369,370,2019-05-01,primavera,2019,mayo,0,3,1,1,10.899153,13.92060,52.4167,8.709129,140,3132,3272
332,333,2018-11-29,otoño,2018,noviembre,0,4,1,2,18.791653,22.56875,83.0833,17.292164,96,2818,2914
269,270,2018-09-27,otoño,2018,septiembre,0,4,1,2,26.103347,28.72625,88.5417,7.917457,477,3643,4120
448,449,2019-03-25,primavera,2019,marzo,0,1,0,2,17.937500,21.87440,88.0833,14.791925,1532,3464,4996
374,375,2019-10-01,otoño,2019,octubre,0,2,1,1,12.656536,15.94130,64.6522,12.565984,173,3425,3598


In [73]:
def cambiar_dia(columna):

    """Recibe una columna tipo datetime y devuelve el día de la semana que corresponde."""
    
    dia = columna.strftime("%w")
    
    if dia == "0":
        return "domingo" 
    elif dia == "1":
        return "lunes"
    elif dia == "2":
        return "martes"
    elif dia == "3":
        return "miércoles"
    elif dia == "4":
        return "jueves"
    elif dia == "5":
        return "viernes"
    else:
        return "sábado"

In [74]:
df["dia_semana"] = df["fecha"].apply(cambiar_dia)

In [75]:
df.index

RangeIndex(start=0, stop=730, step=1)

In [76]:
def festivos_DC(col):

    """Devuelve los festivos de la ciudad de Washington DC.
    Args: fecha que queremos analizar sin es festiva o no
    Return: string con 'sí' o 'no' en función de si es festivo o no.
    """

    festivo = holidays.US(state = 'DC')

    if col in festivo:
        return 'sí'
    else:
        return 'no'

In [77]:
df['festivo'] = df['fecha'].apply(festivos_DC)

In [104]:
def laborales(col):

    """Devuelve los días laborales de la ciudad de Washington DC (en función de si son festivos o fin de semana, o días no festivos entre semana)
    Args: fecha que queremos analizar sin es laboral o no.
    Return: string con 'sí' o 'no' en función de si es laboral o no.
    """

    dia = col.strftime("%w")
    festivo = holidays.US(state = 'DC')
    
    if col in festivo:
        return "no"
    elif dia == "6" or dia == "0":
        return "no"
    else:
        return "sí"

In [105]:
df['laboral'] = df['fecha'].apply(laborales)

In [81]:
def cambiar_clima(dia):

    """Cambia la codificación de la columna clima de números a texto descriptivo
    Args: registro que queremos cambiar
    Return: registro modificado
    """
    
    if dia == 1:
        return "cielo despejado" 
    elif dia == 2:
        return " niebla"
    elif dia == 3:
        return "precipitaciones ligeras"
    else:
        return "precipitaciones abundantes"

In [84]:
df['clima'] = df['clima'].map(cambiar_clima)

In [91]:
lista_redondeo = ["temperatura", "sensacion_termica", "humedad", "viento"]

for col in lista_redondeo:
    df[col] = df[col].apply(lambda elemento: round(elemento, 1))

In [121]:
df.head()

,fecha,estacion,año,mes,festivo,dia_semana,laboral,clima,temperatura,sensacion_termica,humedad,viento,casual,registrado,total
0,2018-01-01,invierno,2018,enero,sí,lunes,no,niebla,14.1,18.2,80.6,10.7,331,654,985
1,2018-02-01,invierno,2018,febrero,no,jueves,sí,niebla,14.9,17.7,69.6,16.7,131,670,801
2,2018-03-01,invierno,2018,marzo,no,jueves,sí,cielo despejado,8.1,9.5,43.7,16.6,120,1229,1349
3,2018-04-01,primavera,2018,abril,no,domingo,no,cielo despejado,8.2,10.6,59.0,10.7,108,1454,1562
4,2018-05-01,primavera,2018,mayo,no,martes,sí,cielo despejado,9.3,11.5,43.7,12.5,82,1518,1600


In [119]:
#Hemos observado que la columna momento no nos aporta nada de información relevante para nuestro análisis, procedemos a eliminarla.

df.drop('momento', axis = 1, inplace = True)